### Required Libraries

In [24]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [25]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

In [26]:
def map_weight(value):
    if (value >= '2022-10') and (value <= '2023-03'):
        return 2
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


### Drop Null target records

In [5]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [28]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [29]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [30]:
model_df1.shape, oot_df1.shape

((101984, 2832), (17343, 2832))

In [31]:
features_list=[
 'distinct_merchant',
 'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
 'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
 'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'max_one_time_spent',
 'amt_first_android_txn',
 'MAX(pg_success_raw_data.amount)',
 'top_payment_mode_count',
 'amt_first_cc_txn',
               'COUNT(raw WHERE amount_category = 6)/COUNT(raw)#ever',
               'ENTROPY(raw.mode_cleaned)#720',
               'COUNT(pg_success_raw_data WHERE revised_merchant_category = bill_pay)',
               'YEAR(first_pg_failed_raw_data_time)',
 'month_year',
    'Weight_vintage',
 '60_in_15mob']

In [32]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [33]:
oot_df = oot_df1[features_list]

In [34]:
model_df.shape, oot_df.shape

((101984, 16), (17343, 16))

In [35]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [36]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [37]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [38]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [39]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3162266/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [40]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3162266/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [41]:
X_oot.shape,y_oot.shape

((17343, 14), (17343,))

### Split Model dataset into Train and Test

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [43]:
X_train.shape, X_test.shape

((61190, 14), (40794, 14))

In [44]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 14) (12238, 14) (40794, 14)


In [45]:
# Get Vinateg weight values 
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [46]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [47]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [48]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.005,
    'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 4.7,
    'alpha': 1.8,
    'random_state': 42  
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54290	eval-auc:0.52823
[10]	train-auc:0.56656	eval-auc:0.54334
[20]	train-auc:0.56856	eval-auc:0.54566
[30]	train-auc:0.56974	eval-auc:0.54573
[40]	train-auc:0.56905	eval-auc:0.54547
[50]	train-auc:0.56990	eval-auc:0.54620
[60]	train-auc:0.56972	eval-auc:0.54653
[70]	train-auc:0.57037	eval-auc:0.54766
[80]	train-auc:0.57020	eval-auc:0.54757
[90]	train-auc:0.57025	eval-auc:0.54719
[100]	train-auc:0.57076	eval-auc:0.54847
[110]	train-auc:0.57062	eval-auc:0.54795
[120]	train-auc:0.57097	eval-auc:0.54771
[130]	train-auc:0.57085	eval-auc:0.54730
[140]	train-auc:0.57106	eval-auc:0.54770
[149]	train-auc:0.57128	eval-auc:0.54761


In [49]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.14, KS: 0.11
Eval AUC: 0.55, Gini: 0.10, KS: 0.08
Test AUC: 0.56, Gini: 0.12, KS: 0.10
OOT AUC: 0.56, Gini: 0.11, KS: 0.09


In [27]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.57, Gini: 0.1489, KS: 0.1160
Test AUC: 0.56, Gini: 0.1232, KS: 0.0977
OOT AUC: 0.56, Gini: 0.1199, KS: 0.0993


In [50]:
# Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_PG_Train_Pred_Vin.csv',index=False)
Eval.to_csv('LeoPayu_PG_Eval_Pred_Vin.csv',index=False)
Test.to_csv('LeoPayu_PG_Test_Pred_Vin.csv',index=False)
Oot.to_csv('LeoPayu_PG_Oot_Pred_Vin.csv',index=False)


In [29]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
4                                  max_one_time_spent       137.0
2   MEAN(raw.amount WHERE device_os_category = Win...       135.0
10                      ENTROPY(raw.mode_cleaned)#720       129.0
6                     MAX(pg_success_raw_data.amount)        93.0
12                YEAR(first_pg_failed_raw_data_time)        83.0
5                               amt_first_android_txn        67.0
11  COUNT(pg_success_raw_data WHERE revised_mercha...        65.0
1   SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...        61.0
9   COUNT(raw WHERE amount_category = 6)/COUNT(raw...        40.0
3   NUM_UNIQUE(pg_success_raw_data.month_end_start...        34.0
0                                   distinct_merchant        29.0
8                                    amt_first_cc_txn        21.0
7                              top_payment_mode_count        21.0


In [30]:
feature_importance_df

Feature  Importance
4                                  max_one_time_spent       137.0
2   MEAN(raw.amount WHERE device_os_category = Win...       135.0
10                      ENTROPY(raw.mode_cleaned)#720       129.0
6                     MAX(pg_success_raw_data.amount)        93.0
12                YEAR(first_pg_failed_raw_data_time)        83.0
5                               amt_first_android_txn        67.0
11  COUNT(pg_success_raw_data WHERE revised_mercha...        65.0
1   SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...        61.0
9   COUNT(raw WHERE amount_category = 6)/COUNT(raw...        40.0
3   NUM_UNIQUE(pg_success_raw_data.month_end_start...        34.0
0                                   distinct_merchant        29.0
8                                    amt_first_cc_txn        21.0
7                              top_payment_mode_count        21.0

In [31]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
9          0.164652         0.143432        0.158739        0.139208   2399   
8          0.129486         0.138428        0.139204        0.131766  12287   
7          0.136670         0.125160        0.131763        0.121826   4895   
6          0.106027         0.120295        0.121826        0.118823   4895   
5          0.094178         0.117488        0.118822        0.116251   4895   
4          0.102349         0.114795        0.116250        0.113247   4895   
3          0.087028         0.111602        0.113247        0.110020   4895   
2          0.077222         0.108542        0.110020        0.107069   4895   
1          0.069853         0.105005        0.107068        0.099641   4896   

        label_good  label_bad  
Decile                         
9             2004        395  
8            10696       1591  
7             4226        669  
6             4376        519  
5             4434        461  
4             4394        501  
3             4469        426  
2             4517        378  
1             4554        342

In [32]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
9          0.153958         0.143419        0.157633        0.139208   2072   
8          0.132304         0.138462        0.139204        0.132160  10166   
7          0.126471         0.125157        0.132156        0.121757   4080   
6          0.107624         0.120242        0.121757        0.118750   4079   
5          0.103702         0.117399        0.118750        0.116133   4079   
4          0.097549         0.114689        0.116133        0.113189   4080   
3          0.093160         0.111605        0.113189        0.110036   4079   
2          0.095612         0.108574        0.110035        0.107116   4079   
1          0.074265         0.105089        0.107116        0.099697   4080   

        label_good  label_bad  
Decile                         
9             1753        319  
8             8821       1345  
7             3564        516  
6             3640        439  
5             3656        423  
4             3682        398  
3             3699        380  
2             3689        390  
1             3777        303

In [53]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
9          0.146435         0.150888        0.158783        0.147927   1038   
8          0.123409         0.146961        0.147923        0.140820   4165   
7          0.117647         0.138406        0.140817        0.136781   1734   
6          0.102653         0.135809        0.136780        0.134903   1734   
5          0.108934         0.134212        0.134903        0.133536   1735   
4          0.093426         0.132778        0.133535        0.131907   1734   
3          0.091696         0.130688        0.131905        0.129500   1734   
2          0.087082         0.128662        0.129500        0.127933   1734   
1          0.074352         0.127136        0.127932        0.125157   1735   

        label_good  label_bad  
Decile                         
9              886        152  
8             3651        514  
7             1530        204  
6             1556        178  
5             1546        189  
4             1572        162  
3             1575        159  
2             1583        151  
1             1606        129

In [52]:
#Save the Model
xgb_model.save_model('LeoPayu_PG_60_in_15mob_Vin.json')
print('Model Saved')

Model Saved
